In [ ]:

!pip install --quiet transformers


     |████████████████████████████████| 4.0 MB 7.3 MB/s 
     |████████████████████████████████| 895 kB 33.1 MB/s 
     |████████████████████████████████| 77 kB 1.9 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.1 MB/s 


In [ ]:
import os
from datetime import datetime
import logging

import pandas as pd
# from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

# from utils import load_data, clean_unnecessary_spaces


# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.ERROR)

In [ ]:
import warnings
import json
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from random import shuffle

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from transformers import BartTokenizer
import numpy as np
from joblib import dump


In [ ]:
!pip install --quiet utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!git clone https://github.com/AshOlogn/Paragraph-level-Simplification-of-Medical-Texts.git

Cloning into 'Paragraph-level-Simplification-of-Medical-Texts'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 149 (delta 67), reused 124 (delta 50), pack-reused 0
Receiving objects: 100% (149/149), 20.02 MiB | 8.95 MiB/s, done.
Resolving deltas: 100% (67/67), done.
Checking out files: 100% (46/46), done.


In [ ]:
cd drive/My\ Drive/NLP-Written/Paragraph-level-Simplification-of-Medical-Texts/


/content/drive/My Drive/NLP-Written/Paragraph-level-Simplification-of-Medical-Texts


In [ ]:
df=pd.read_json('data/data_final_1024.json')

In [ ]:
df=pd.read_csv('data.csv')
df = df.drop(labels=2267, axis=0)

In [ ]:
df.tail()

,Unnamed: 0,source_text,target_text,source_id,target_id
2262,2262,involved medical researchers at the university...,doctors and experts at the university of texas...,0,1
2263,2263,her condition is similar to many other conditi...,"her condition is like some others , especially...",0,1
2264,2264,velazquez has a condition that is so rare that...,only two other people are known to have the co...,0,1
2265,2265,two polio vaccines are used throughout the wor...,two polio vaccines are used throughout the wor...,0,1
2266,2266,"therefore , interruption of person to person t...","therefore , interruption of person to person t...",0,1


In [ ]:
df['abs_id']=0
df['pls_id']=1
df_abs=df[['abstract','abs_id']]
df_pls=df[['pls','pls_id']]
df_full=pd.DataFrame(np.vstack([df_abs, df_pls]), columns=['text','id'])


In [ ]:
df['source_id']=0
df['target_id']=1
df_abs=df[['source_text','source_id']]
df_pls=df[['target_text','target_id']]
df_full=pd.DataFrame(np.vstack([df_abs, df_pls]), columns=['text','id'])


In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')
# tokenizer=tokenizer.to(device)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='facebook/bart-large-xsum', vocab_size=50265, model_max_len=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [ ]:
def make_vector(text):
    token_ids = tokenizer.encode(text)[1:-1]
    count_vector = np.zeros(tokenizer.vocab_size, dtype=np.int16)
    for ID in token_ids:
        count_vector[ID] += 1
    return count_vector

In [ ]:
np.random.shuffle(df_full.values)


In [ ]:
df_full.shape

(4534, 2)

In [ ]:
df_small=df_full[:3000]

In [ ]:
df_small['vect']=df_full.text.apply(make_vector)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_small.head()

,text,id,vect
0,"on june 12 , 1994 , calloway suffered a severe...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"the opposite condition , when body temperature...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,while the resuscitator is being used to save m...,1,"[0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, ..."
3,when he was recovering in the hospital from ba...,1,"[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 2, ..."
4,levamisole has been used in humans to treat pa...,0,"[0, 0, 0, 0, 0, 0, 0, 1, 3, 1, 0, 2, 0, 1, 0, ..."


In [ ]:
split_df=pd.DataFrame(df_small['vect'].to_list())

In [ ]:
df_token=pd.concat([df_full,split_df],axis=1)

In [ ]:
split_df=pd.DataFrame(normalize(split_df))

In [ ]:
split_df.head()

,0,1,2,3,4,5,6,7,8,9,...,50255,50256,50257,50258,50259,50260,50261,50262,50263,50264
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.188982,0.188982,0.0,0.188982,0.188982,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.140028,0.0,0.140028,0.140028,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.130189,0.390567,0.130189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y=df_small[:3000]['id']
model = LogisticRegression(max_iter=100)
model.fit(split_df[:3000],list(y))

LogisticRegression()

In [ ]:
split_df[:150],list(y)

In [ ]:
dump(model, 'logisticregression.joblib')

['logisticregression.joblib']

In [ ]:
def get_vocab(tokenizer):
    tokens = [tokenizer.decode([i], clean_up_tokenization_spaces=False) for i in range(tokenizer.vocab_size)]
    return tokens

In [ ]:
len(list(sorted_weights))

50242

In [ ]:
tokenizer.vocab_size

50265

In [ ]:
len(dfW)

50265

In [ ]:
weights = np.squeeze(model.coef_, axis=0).tolist()
sorted_weights=list(zip(range(tokenizer.vocab_size),weights))
dfW=pd.DataFrame(sorted_weights)
dfW.to_csv('weights.csv')

In [ ]:
sorted_weights=list(zip(vocab,weights))
dfW=pd.DataFrame(sorted_weights)
dfW.to_csv('weights_token.csv')

In [ ]:
vocab = get_vocab(tokenizer)
weights = np.squeeze(model.coef_, axis=0).tolist()

sorted_weights = filter(lambda x: len(x[1].strip()) > 0, zip(range(tokenizer.vocab_size), vocab, weights))
sorted_weights = list(sorted(sorted_weights, key=lambda x: x[2]))

with open('bart_freq_normalized_ids.csv','w') as f:
    for ID, word, weight in sorted_weights:
        f.write(f'{ID}, {weight}\n')

with open('bart_freq_normalized_tokens.txt','w') as f:
    for ID, word, weight in sorted_weights:
        f.write(f'{word} {weight}\n')


In [ ]:
y_test=list(df_small[2000:]['id'])

In [ ]:
predictions = model.predict(split_df[2000:])
accuracy = accuracy_score(y_test, predictions)

In [ ]:
sorted_weights_df=pd.DataFrame(sorted_weights,columns=['id','token','weights'])

In [ ]:
sorted_weights_df=sorted_weights_df.sort_values(by=['weights'])

In [ ]:
sorted_weights_df.to_csv('sorted_weights.csv')

In [ ]:
accuracy


0.71

In [ ]:
dfW=pd.read_csv('bart_freq_normalized_ids.csv',header=0,index_col=False)

In [ ]:
dfW.to_numpy().shape

(50241, 2)

In [ ]:
dfW.to_numpy()

array([[ 25.        ,  -1.11468097],
       [ 19.        ,  -1.01009528],
       [484.        ,  -1.00712843],
       ...,
       [621.        ,   1.51840536],
       [ 26.        ,   1.9703849 ],
       [479.        ,   2.51037834]])

In [ ]:
textstat.automated_readability_index(df['pls'][0])


14.7

In [ ]:
!pip install --quiet transformers
!pip install --quiet callbacks

In [ ]:
!pip install --quiet utils